In [1]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
# pip install pyLDAvis

In [2]:
#plotly.offline doesn't push your charts to the clouds
# import plotly.offline as pyo
# #allows us to create the Data and Figure objects
# from plotly.graph_objs import *
# #plotly.plotly pushes your charts to the cloud  
# # import chart_studio.plotly as py
# import plotly.figure_factory as ff
# # import plotly.express as px

# import matplotlib.pyplot as plt
# %matplotlib inline

# # import cufflings to easily plot pandas data frames
# import cufflinks as cf
# # work with cufflinks offline and set its theme
# cf.go_offline()
# cf.set_config_file(theme='pearl')

import pandas as pd
import numpy as np

# from utils import *

import os
import gc
import json
import re
import gzip, pickle
import gc
%matplotlib inline

In [3]:
def print_full(x):
    pd.set_option('display.max_colwidth', -1)
    return x

def reset():
    pd.reset_option('display.max_colwidth')

In [13]:
df_askmen = pd.read_json("askMen2018_clean.json", orient='table')[['body', 'gender']]
df_askmen

,body,gender
0,I've never met an attractive woman that (given...,m
1,Ouch. Being ghosted on really sucks. I can't s...,m
2,Yeah exactly. Like I have no specific thoughts...,m
3,I touch my boobs then smell my hands. I have n...,f
4,"I’m on a college campus, it sticks around unti...",m
...,...,...
601692,"Get a savings account, and deliberately put mo...",m
601693,It is working so far,m
601694,Last week tonight with John Oliver. I was very...,m
601695,You’re right.,m


In [16]:
df_askwomen = pd.read_json("askWomen2018_clean.json", orient='table')[['body', 'gender']]
df_askwomen

[f, m]
Categories (2, object): [f, m]

In [18]:
df_askwomen

,body,gender
0,Every year my brother has a low let get togeth...,f
1,Every year my brother has a low let get togeth...,f
2,Every year my brother has a low let get togeth...,f
3,Every year my brother has a low let get togeth...,f
4,Every year my brother has a low let get togeth...,f
...,...,...
518982,You should look into medical anthropology!,f
518983,15ish hours with a loud ass cat in my back sea...,f
518984,Porn isn't anything like real sex -- thank hea...,f
518985,"Ramen noodles. (Ah, who are we kidding? Those ...",f


In [15]:
df_relation = pd.read_json("relationship2018_clean.json", orient='table')[['body', 'gender']]
df_relation

,body,gender
0,I think you might feel less guilty if you talk...,m
1,I doubt she's trying to escape the situation. ...,m
2,Are you going to address my first question?\n\...,f
3,why are you even considering not giving your s...,f
4,The above submission has been **removed** beca...,m
...,...,...
704501,"Hey, no real in depth advice about how to feel...",f
704502,"Hey, I get it. My good friends have had Europe...",f
704503,"If he's that broke and prideful, he should sta...",f
704504,"Interestingly enough, it doesn't matter if you...",f


In [9]:
df_relation.head(10)

,body,gender
0,I think you might feel less guilty if you talk...,m
1,I doubt she's trying to escape the situation. ...,m
2,Are you going to address my first question?\n\...,f
3,why are you even considering not giving your s...,f
4,The above submission has been **removed** beca...,m
5,"As I said in my original comment, please take ...",f
6,Do you have a significantly better relationshi...,f
7,&gt; You have a CLEAR hang-up on your first gi...,f
8,And you’re assuming things on the basis that s...,f
9,Kids are jerks especially teenagers. Let it go.,f


In [26]:
df_askwomen.drop_duplicates(keep='first',inplace=True)
df_askmen.drop_duplicates(keep='first',inplace=True)
df_relation.drop_duplicates(keep='first',inplace=True)

In [29]:
df_askmen

,body,gender
0,I've never met an attractive woman that (given...,m
1,Ouch. Being ghosted on really sucks. I can't s...,m
2,Yeah exactly. Like I have no specific thoughts...,m
3,I touch my boobs then smell my hands. I have n...,f
4,"I’m on a college campus, it sticks around unti...",m
...,...,...
601692,"Get a savings account, and deliberately put mo...",m
601693,It is working so far,m
601694,Last week tonight with John Oliver. I was very...,m
601695,You’re right.,m


In [72]:
df_askwomen.reset_index(drop=True,inplace=True)
df_askmen.reset_index(drop=True,inplace=True)
df_relation.reset_index(drop=True,inplace=True)

In [73]:
df_askwomen

,body,gender
0,Every year my brother has a low let get togeth...,f
1,I know cargo shorts aren’t supposed to be in s...,f
2,Happy birthday!!!,f
3,"Use painter's tape to protect areas, not to cu...",f
4,Playing video games and drinking Kontinis with...,f
...,...,...
498539,You should look into medical anthropology!,f
498540,15ish hours with a loud ass cat in my back sea...,f
498541,Porn isn't anything like real sex -- thank hea...,f
498542,"Ramen noodles. (Ah, who are we kidding? Those ...",f


In [71]:
df_askmen

,body,gender
0,I've never met an attractive woman that (given...,m
1,Ouch. Being ghosted on really sucks. I can't s...,m
2,Yeah exactly. Like I have no specific thoughts...,m
3,I touch my boobs then smell my hands. I have n...,f
4,"I’m on a college campus, it sticks around unti...",m
...,...,...
601692,"Get a savings account, and deliberately put mo...",m
601693,It is working so far,m
601694,Last week tonight with John Oliver. I was very...,m
601695,You’re right.,m


In [32]:
df_askmen.memory_usage(deep=True).sum()/1024**2

187.9977798461914

In [33]:
gc.collect()

52

In [35]:
import spacy

In [36]:
# this is to use the default english language model for tokeniziation, pos, etc.
# but use 'en_vectors_web_lg' vectors --> over 1 million 300-dimensional Glove vectors
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# spacy.load('en_vectors_web_lg', vocab=nlp.vocab)

In [37]:
nlp.vocab.vectors.shape

(0, 0)

In [38]:
from spacy.lang.en.stop_words import STOP_WORDS

In [39]:
STOP_WORDS

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

## Beginning Text Preprocessing

We'd like now to see which terms are more frequent in the corpus. To do this we need to clean and tokenize our corpus first 

#### Parsing Tweets Using Spacy Pipeline

In [40]:
sent = "Joanna is Living in China, Studying machine learning? in Peking University! I'm happy with this. That's great!! What a great field of study that is 133. don't teach machine learning on your own. You can't do this"

spacy_sent = nlp(sent)

In [41]:
type(sent)

str

In [42]:
spacy_sent

Joanna is Living in China, Studying machine learning? in Peking University! I'm happy with this. That's great!! What a great field of study that is 133. don't teach machine learning on your own. You can't do this

In [43]:
type(spacy_sent)

spacy.tokens.doc.Doc

In [44]:
sent[0]

'J'

In [45]:
spacy_sent[0]

Joanna

In [46]:
type(spacy_sent[0])

spacy.tokens.token.Token

In [47]:
spacy_sent[0].is_stop

False

In [48]:
[t.lemma_ for t in spacy_sent
 if not t.is_punct]

['Joanna',
 'be',
 'live',
 'in',
 'China',
 'study',
 'machine',
 'learning',
 'in',
 'Peking',
 'University',
 '-PRON-',
 'be',
 'happy',
 'with',
 'this',
 'that',
 'be',
 'great',
 'what',
 'a',
 'great',
 'field',
 'of',
 'study',
 'that',
 'be',
 '133',
 'do',
 'not',
 'teach',
 'machine',
 'learn',
 'on',
 '-PRON-',
 'own',
 '-PRON-',
 'can',
 'not',
 'do',
 'this']

In [49]:
[t.lemma_ for t in spacy_sent 
 if not t.lemma_ in STOP_WORDS 
 and t.is_alpha  
 and not t.lemma_ == '-PRON-']

['Joanna',
 'live',
 'China',
 'study',
 'machine',
 'learning',
 'Peking',
 'University',
 'happy',
 'great',
 'great',
 'field',
 'study',
 'teach',
 'machine',
 'learn']

In [50]:
" ".join([t.lemma_ for t in spacy_sent 
           if not t.lemma_ in STOP_WORDS 
           and t.is_alpha  
           and not t.lemma_ == '-PRON-'])

'Joanna live China study machine learning Peking University happy great great field study teach machine learn'

In [51]:
# using generator
sent_gen = (t.lemma_ for t in spacy_sent 
           if not t.lemma_ in STOP_WORDS 
           and t.is_alpha  
           and not t.lemma_ == '-PRON-')

In [52]:
for token in sent_gen:
    print(token)

Joanna
live
China
study
machine
learning
Peking
University
happy
great
great
field
study
teach
machine
learn


In [53]:
# next(sent_gen)

In [54]:
[t.lemma_ for t in spacy_sent 
 if not t.lemma_ in STOP_WORDS 
 if not t.is_punct 
 and not t.like_num 
 and not t.lemma_ == '-PRON-']

['Joanna',
 'live',
 'China',
 'study',
 'machine',
 'learning',
 'Peking',
 'University',
 'happy',
 'great',
 'great',
 'field',
 'study',
 'teach',
 'machine',
 'learn']

In [55]:
for token in spacy_sent:
    print(token.text, token.pos_, token.is_stop, token.is_punct)

Joanna PROPN False False
is AUX True False
Living VERB False False
in ADP True False
China PROPN False False
, PUNCT False True
Studying VERB False False
machine NOUN False False
learning NOUN False False
? PUNCT False True
in ADP True False
Peking PROPN False False
University PROPN False False
! PUNCT False True
I PRON True False
'm AUX True False
happy ADJ False False
with ADP True False
this DET True False
. PUNCT False True
That DET True False
's AUX True False
great ADJ False False
! PUNCT False True
! PUNCT False True
What DET True False
a DET True False
great ADJ False False
field NOUN False False
of ADP True False
study NOUN False False
that DET True False
is AUX True False
133 NUM False False
. PUNCT False True
do AUX True False
n't PART True False
teach VERB False False
machine NOUN False False
learning VERB False False
on ADP True False
your DET True False
own ADJ True False
. PUNCT False True
You PRON True False
ca VERB True False
n't PART True False
do AUX True False
this 

In [56]:
for ent in spacy_sent.ents:
    print(ent.text, ent.label_)

In [57]:
spacy.explain('GPE')

'Countries, cities, states'

In [58]:
spacy.explain('CARDINAL')

'Numerals that do not fall under another type'

In [59]:
spacy.explain('ORG')

'Companies, agencies, institutions, etc.'

### Arithmetic with Word Vectors

In [60]:
london = nlp('London').vector
paris = nlp('Paris').vector
uk = nlp('Britain').vector
france = nlp('France').vector

In [61]:
res = london - uk + france
res

array([-1.9894615e+00, -4.4303656e+00,  1.4506032e+00,  2.0614474e+00,
        1.8862536e+00,  5.5113354e+00,  2.4421766e+00,  1.8836199e+00,
        2.5335824e+00, -2.7786801e+00,  6.0046616e+00, -2.2625604e+00,
        2.9566431e+00, -1.0319500e+00, -1.1317499e+00, -1.7165416e-01,
       -3.2619135e+00,  1.9313313e+00,  3.0805292e+00, -3.8252678e+00,
       -9.5556867e-01, -1.4942904e+00,  3.0041740e+00, -1.0425816e+00,
       -1.6118513e+00, -3.8741512e+00,  1.4092798e+00, -2.8371730e+00,
       -2.9041648e-02, -2.6587942e+00, -1.3275492e+00, -2.6508548e+00,
       -4.5881834e+00,  1.8748331e-01,  1.7193993e+00, -2.1163092e+00,
        3.4957612e+00, -1.0249393e+00, -1.1622918e+00, -8.1404269e-01,
        3.5516284e+00, -9.4727802e-01,  3.2907836e+00, -6.6551037e+00,
       -3.8677740e-01,  7.2591335e-01,  4.2703438e-01, -4.8039341e-01,
        1.9079071e+00,  1.0688066e+00, -3.2319837e+00, -3.3663535e+00,
       -1.6425936e+00, -2.5447140e+00, -1.8900532e+00,  2.3669238e+00,
      

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
names = ['london', 'paris', 'uk', 'france', 'res']

In [64]:
res_df = pd.DataFrame(cosine_similarity([london, paris, uk, france, res]), index=names, columns=names)

In [65]:
res_df

,london,paris,uk,france,res
london,1.000000,0.706032,0.807281,0.814918,0.850428
paris,0.706032,1.000000,0.680290,0.760310,0.665536
uk,0.807281,0.680290,1.000000,0.809359,0.521372
france,0.814918,0.760310,0.809359,1.000000,0.852678
res,0.850428,0.665536,0.521372,0.852678,1.000000


#### Processing many documents at once 

In [74]:
df_askmen['body'].head()

0    I've never met an attractive woman that (given...
1    Ouch. Being ghosted on really sucks. I can't s...
2    Yeah exactly. Like I have no specific thoughts...
3    I touch my boobs then smell my hands. I have n...
4    I’m on a college campus, it sticks around unti...
Name: body, dtype: object

In [75]:
df_askwomen['body'].head()

0    Every year my brother has a low let get togeth...
1    I know cargo shorts aren’t supposed to be in s...
2                                   Happy birthday!!! 
3    Use painter's tape to protect areas, not to cu...
4    Playing video games and drinking Kontinis with...
Name: body, dtype: object

In [76]:
df_relation['body'].head()

0    I think you might feel less guilty if you talk...
1    I doubt she's trying to escape the situation. ...
2    Are you going to address my first question?\n\...
3    why are you even considering not giving your s...
4    The above submission has been **removed** beca...
Name: body, dtype: object

#### Cleaning

In [77]:
STOP_WORDS = STOP_WORDS.difference({'not', 'never', 'neither', 'nor'}) # don't include "not" in the stop words

In [78]:
gc.collect()

286

In [83]:
df_askwomen

,body,gender
0,Every year my brother has a low let get togeth...,f
1,I know cargo shorts aren’t supposed to be in s...,f
2,Happy birthday!!!,f
3,"Use painter's tape to protect areas, not to cu...",f
4,Playing video games and drinking Kontinis with...,f
...,...,...
498539,You should look into medical anthropology!,f
498540,15ish hours with a loud ass cat in my back sea...,f
498541,Porn isn't anything like real sex -- thank hea...,f
498542,"Ramen noodles. (Ah, who are we kidding? Those ...",f


In [80]:
%%time
# run one time and then change to if False
if True:
    
    df_askmen['body_clean'] = [[t.lemma_ for t in text # get the lemma form of the word
                                if t.is_alpha     # remove punctuations and non alphabetical tokens
                                and not t.lemma_ in STOP_WORDS # remove stop words
                                and not t.lemma_ == '-PRON-'] for text in nlp.pipe(df_askmen['body'], n_process=-1)]
    
    # save to desk
    df_askmen.to_json("askMen2018_processed.json", orient='table')

CPU times: user 9min 17s, sys: 48.3 s, total: 10min 5s
Wall time: 10min 20s


In [84]:
%%time
# run one time and then change to if False
if True:
    
    df_askwomen['body_clean'] = [[t.lemma_ for t in text # get the lemma form of the word
                                if t.is_alpha     # remove punctuations and non alphabetical tokens
                                and not t.lemma_ in STOP_WORDS # remove stop words
                                and not t.lemma_ == '-PRON-'] for text in nlp.pipe(df_askwomen['body'], n_process=-1)]
    
    # save to desk
    df_askwomen.to_json("askWomen2018_processed.json", orient='table')

CPU times: user 9min 11s, sys: 52.9 s, total: 10min 4s
Wall time: 10min 14s


In [85]:
%%time
# run one time and then change to if False
if True:
    
    df_relation['body_clean'] = [[t.lemma_ for t in text # get the lemma form of the word
                                if t.is_alpha     # remove punctuations and non alphabetical tokens
                                and not t.lemma_ in STOP_WORDS # remove stop words
                                and not t.lemma_ == '-PRON-'] for text in nlp.pipe(df_relation['body'], n_process=-1)]
    
    # save to desk
    df_relation.to_json("relationship2018_processed.json", orient='table')

CPU times: user 19min 2s, sys: 3min 23s, total: 22min 26s
Wall time: 23min 17s


In [86]:
df_askmen = pd.read_json("askMen2018_processed.json", orient='table')

In [92]:
df_askmen

,body,gender,body_clean
0,I've never met an attractive woman that (given...,m,"[never, meet, attractive, woman, single, time,..."
1,Ouch. Being ghosted on really sucks. I can't s...,m,"[ouch, ghost, suck, never, think, respectable,..."
2,Yeah exactly. Like I have no specific thoughts...,m,"[yeah, exactly, like, specific, thought, annoy..."
3,I touch my boobs then smell my hands. I have n...,f,"[touch, boob, smell, hand, idea, odd, not, lik..."
4,"I’m on a college campus, it sticks around unti...",m,"[college, campus, stick, mid, February, terrib..."
...,...,...,...
564097,"Get a savings account, and deliberately put mo...",m,"[saving, account, deliberately, money, month, ..."
564098,It is working so far,m,"[work, far]"
564099,Last week tonight with John Oliver. I was very...,m,"[week, tonight, John, Oliver, vocal, hate, pol..."
564100,You’re right.,m,[right]


In [88]:
df_askmen.memory_usage(deep=True).sum()/1024**2

279.03429985046387

In [89]:
df_askwomen = pd.read_json("askWomen2018_processed.json", orient='table')
df_relation = pd.read_json("relationship2018_processed.json", orient='table')

In [90]:
df_askwomen

,body,gender,body_clean
0,Every year my brother has a low let get togeth...,f,"[year, brother, low, let, PJs, beer, lot, snac..."
1,I know cargo shorts aren’t supposed to be in s...,f,"[know, cargo, short, suppose, style, anymore, ..."
2,Happy birthday!!!,f,"[happy, birthday]"
3,"Use painter's tape to protect areas, not to cu...",f,"[use, painter, tape, protect, area, not, cut, ..."
4,Playing video games and drinking Kontinis with...,f,"[play, video, game, drink, Kontinis, fresh, li..."
...,...,...,...
498539,You should look into medical anthropology!,f,"[look, medical, anthropology]"
498540,15ish hours with a loud ass cat in my back sea...,f,"[hour, loud, ass, cat, seat, not, miss, drive,..."
498541,Porn isn't anything like real sex -- thank hea...,f,"[porn, like, real, sex, thank, heavens]"
498542,"Ramen noodles. (Ah, who are we kidding? Those ...",f,"[ramen, noodle, ah, kid, thing, delicious, mat..."


In [91]:
df_relation

,body,gender,body_clean
0,I think you might feel less guilty if you talk...,m,"[think, feel, guilty, talk, wife, know, little..."
1,I doubt she's trying to escape the situation. ...,m,"[doubt, try, escape, situation, suddenly, comp..."
2,Are you going to address my first question?\n\...,f,"[address, question, guy, directly, ask, fiancé..."
3,why are you even considering not giving your s...,f,"[consider, not, sister, ticket, adult, need, l..."
4,The above submission has been **removed** beca...,m,"[submission, remove, not, appear, contain, sor..."
...,...,...,...
696651,"Hey, no real in depth advice about how to feel...",f,"[hey, real, depth, advice, feel, normal, miss,..."
696652,"Hey, I get it. My good friends have had Europe...",f,"[hey, good, friend, european, trip, undergrad,..."
696653,"If he's that broke and prideful, he should sta...",f,"[broke, prideful, stay, home]"
696654,"Interestingly enough, it doesn't matter if you...",f,"[interestingly, matter, boundary, place, want,..."


## Topic Modeling

In [108]:
askmen_women

,body,gender,body_clean
3,I touch my boobs then smell my hands. I have n...,f,"[touch, boob, smell, hand, idea, odd, not, lik..."
11,Why would you give a hoot about what other peo...,f,"[hoot, people, think, head, op, male, thought,..."
14,Literally same,f,[literally]
15,A romantic *relationship*?\n\n“Hey I’m not see...,f,"[romantic, relationship, hey, not, nope, Wanna..."
26,Well that is the dream...,f,[dream]
...,...,...,...
564066,"Just FYI, that's probably a deal breaker for a...",f,"[FYI, probably, deal, breaker, sex, worker, di..."
564070,Would absolutely love this. I would never be b...,f,"[Would, absolutely, love, never, bore, thing, ..."
564073,"Good grief, is this how your regular conversat...",f,"[good, grief, regular, conversation, sarcastic..."
564085,That I'm not good enough and I can't make it w...,f,"[not, good, work]"


In [109]:
askmen_men = df_askmen[df_askmen['gender'] == 'm']; assert len(askmen_men['gender'].unique()) == 1

In [110]:
askmen_women = df_askmen[df_askmen['gender'] == 'f']; assert len(askmen_women['gender'].unique()) == 1

In [111]:
askwomen_men = df_askwomen[df_askwomen['gender'] == 'm']; assert len(askwomen_men['gender'].unique()) == 1

In [112]:
askwomen_women = df_askwomen[df_askwomen['gender'] == 'f']; assert len(askwomen_women['gender'].unique()) == 1

In [113]:
relation_men = df_relation[df_relation['gender'] == 'm']; assert len(relation_men['gender'].unique()) == 1

In [114]:
relation_women = df_relation[df_relation['gender'] == 'f']; assert len(relation_women['gender'].unique()) == 1

In [95]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from joblib import load, dump
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
%matplotlib inline


/Users/valentina/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/valentina/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


#### Topic Modeling for The Male Gender

In [96]:
tf_vectorizer_men = CountVectorizer(tokenizer= lambda x: x,
                                    lowercase = False,
                                    max_df = 0.5, 
                                    min_df = 10)
dtm_tf_men = tf_vectorizer_men.fit_transform(askmen_men['body_clean'])

In [97]:
dtm_tf_men.shape

(495872, 21535)

In [98]:
lda_tf_men = LatentDirichletAllocation(n_components=10, max_iter=5, batch_size=2000,
                                       learning_method='online',
                                       n_jobs=-1, verbose=1, random_state=0)

In [99]:
%%time
lda_tf_men.fit(dtm_tf_men)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
CPU times: user 11min 51s, sys: 29.3 s, total: 12min 20s
Wall time: 6min 5s


LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

In [115]:
%%time
pyLDAvis.sklearn.prepare(lda_tf_men, dtm_tf_men, tf_vectorizer_men)

CPU times: user 2min 6s, sys: 5.83 s, total: 2min 12s
Wall time: 7min 53s


/Users/valentina/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.158597  0.051943       1        1  23.972379
1     -0.085113 -0.092260       2        1  17.810020
0     -0.086009 -0.137881       3        1  12.757775
2     -0.198092  0.196099       4        1   8.019080
9     -0.094385  0.113604       5        1   7.791780
6      0.104999 -0.205216       6        1   7.599672
5      0.128252 -0.112941       7        1   6.027384
7     -0.050198 -0.104686       8        1   5.565805
8      0.114135  0.122420       9        1   5.241757
3      0.325008  0.168918      10        1   5.214348, topic_info=      Category          Freq    Term         Total  loglift  logprob
13675  Default  40941.000000     man  40941.000000  30.0000  30.0000
10733  Default  40490.000000  friend  40490.000000  29.0000  29.0000
21334  Default  53791.000000   woman  53791.000000  28.0000  28.0000
21464  Default  38526.000000    year  38526.000000  27.0000  27.0000
15553  Default  66554.000000  people  66554.000000  26.0000  26.0000
...        ...           ...     ...           ...      ...      ...
15680  Topic10   1779.437633   phone   5272.294986   1.8676  -5.3952
14636  Topic10   2039.712944    nice  11989.970350   1.1825  -5.2587
11277  Topic10   1959.522250   great  15349.234212   0.8954  -5.2988
13522  Topic10   1691.364598    love  23224.323438   0.3341  -5.4460
20682  Topic10   1612.884477     use  28835.534858   0.0702  -5.4935

[730 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
142        4  0.998911  America
236        5  0.997177   AskMen
731        1  0.067918      Can
731        4  0.000543      Can
731        8  0.931287      Can
...      ...       ...      ...
21497      2  0.055631    young
21497      3  0.677612    young
21497      4  0.022941    young
21497      8  0.000574    young
21497      9  0.001864    young

[1888 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 10, 7, 6, 8, 9, 4])

#### Topic Modeling for The Female Gender

In [101]:
tf_vectorizer_women = CountVectorizer(tokenizer= lambda x: x,
                                      lowercase = False,
                                      max_df = 0.5, 
                                      min_df = 10)
dtm_tf_women = tf_vectorizer_women.fit_transform(askmen_women['body_clean'])

In [102]:
dtm_tf_women.shape

(68230, 7082)

In [103]:
lda_tf_women = LatentDirichletAllocation(n_components=10, max_iter=5, batch_size=2000,
                                   learning_method='online',
                                   n_jobs=-1, verbose=1, random_state=0)

In [104]:
%%time
lda_tf_women.fit(dtm_tf_women)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
CPU times: user 5.85 s, sys: 1.53 s, total: 7.39 s
Wall time: 1min 12s


LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

In [105]:
import pyLDAvis.sklearn

In [116]:
%%time
pyLDAvis.sklearn.prepare(lda_tf_women, dtm_tf_women, tf_vectorizer_women)

CPU times: user 16.4 s, sys: 182 ms, total: 16.6 s
Wall time: 1min 53s


/Users/valentina/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.036996 -0.068917       1        1  26.653172
2      0.021837 -0.149642       2        1  15.269099
6     -0.021967 -0.122038       3        1  14.050750
7      0.130985  0.084405       4        1  11.308914
8     -0.146813  0.108212       5        1   6.525317
9      0.297912  0.072157       6        1   5.697379
1     -0.074448  0.046940       7        1   5.535199
5     -0.128613  0.233009       8        1   5.227734
4     -0.109753 -0.180292       9        1   5.009869
0     -0.006137 -0.023834      10        1   4.722566, topic_info=     Category          Freq       Term         Total  loglift  logprob
6992  Default  12178.000000      woman  12178.000000  30.0000  30.0000
3986  Default   9379.000000        man   9379.000000  29.0000  29.0000
7048  Default   4703.000000       year   4703.000000  28.0000  28.0000
3852  Default  19633.000000       like  19633.000000  27.0000  27.0000
6368  Default   2193.000000      thank   2193.000000  26.0000  26.0000
...       ...           ...        ...           ...      ...      ...
3050  Topic10    247.913816      great   2129.935201   0.9021  -5.3624
2715  Topic10    263.442706       find   4744.001238   0.1620  -5.3016
3732  Topic10    259.573698       know  10139.010128  -0.6123  -5.3164
533   Topic10    248.649789   actually   3240.909326   0.4853  -5.3594
2089  Topic10    243.675499  different   2749.071115   0.6296  -5.3796

[794 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8         3  0.970543        AW
16       10  0.992099   America
17       10  0.992224  American
25        1  0.012845    AskMen
25        9  0.982641    AskMen
...     ...       ...       ...
7056      4  0.004761       yes
7056      5  0.001058       yes
7056      7  0.457566       yes
7056     10  0.018514       yes
7062      8  0.989530      yoga

[2367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 7, 8, 9, 10, 2, 6, 5, 1])

In [118]:
tf_vectorizer_awmen = CountVectorizer(tokenizer= lambda x: x,
                                    lowercase = False,
                                    max_df = 0.5, 
                                    min_df = 10)
dtm_tf_awmen = tf_vectorizer_awmen.fit_transform(askwomen_men['body_clean'])

In [122]:
lda_tf_awmen = LatentDirichletAllocation(n_components=10, max_iter=5, batch_size=2000,
                                   learning_method='online',
                                   n_jobs=-1, verbose=1, random_state=0)

In [134]:
lda_tf_awmen.fit(dtm_tf_awmen)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

In [124]:
%%time
pyLDAvis.sklearn.prepare(lda_tf_awmen, dtm_tf_awmen, tf_vectorizer_awmen)

CPU times: user 17.7 s, sys: 646 ms, total: 18.3 s
Wall time: 1min 33s


/Users/valentina/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.429608 -0.029255       1        1  38.882460
3     -0.121355 -0.104973       2        1  15.204428
4     -0.084754 -0.099619       3        1   9.488925
6     -0.143450 -0.027046       4        1   7.415418
5      0.412889 -0.000244       5        1   6.898819
2     -0.128054 -0.077715       6        1   6.315037
1     -0.119001 -0.036990       7        1   5.640667
7     -0.104167 -0.030849       8        1   3.893053
0     -0.074658  0.353699       9        1   3.253689
9     -0.067059  0.052991      10        1   3.007505, topic_info=     Category          Freq        Term         Total  loglift  logprob
656   Default  38085.000000      advice  38085.000000  30.0000  30.0000
34    Default  15697.000000    AskWomen  15697.000000  29.0000  29.0000
3501  Default  26123.000000     message  26123.000000  28.0000  28.0000
4441  Default  18421.000000      remove  18421.000000  27.0000  27.0000
3570  Default  18123.000000   moderator  18123.000000  26.0000  26.0000
...       ...           ...         ...           ...      ...      ...
2547  Topic10    168.078021        game   1013.471785   1.7074  -5.3143
3712  Topic10    273.886425         not  28446.878506  -1.1390  -4.8260
5526  Topic10    182.534524         use   2747.145719   0.7927  -5.2318
1790  Topic10    162.078259  definitely    882.462578   1.8094  -5.3507
3774  Topic10    158.991273          oh    968.265900   1.6974  -5.3699

[733 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
14        9  0.982236    Amazon
15        3  0.244707   America
15        8  0.749416   America
34        1  0.495484  AskWomen
34        5  0.504466  AskWomen
...     ...       ...       ...
5787      7  0.047221       yes
5787      8  0.003564       yes
5787      9  0.018710       yes
5787     10  0.343019       yes
5796      6  0.992139       yup

[1882 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 5, 7, 6, 3, 2, 8, 1, 10])

In [121]:
%%time
pyLDAvis.sklearn.prepare(lda_tf_men, dtm_tf_awmen, tf_vectorizer_awmen)

CPU times: user 18.6 s, sys: 2.19 s, total: 20.8 s
Wall time: 1min 36s


/Users/valentina/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.429713  0.029647       1        1  38.883532
3      0.121514  0.105540       2        1  15.242352
4      0.084639  0.100947       3        1   9.520982
6      0.143633  0.027563       4        1   7.382879
5     -0.412900 -0.000666       5        1   6.897670
2      0.127829  0.077604       6        1   6.304026
1      0.119654  0.036790       7        1   5.621417
7      0.102139  0.033524       8        1   3.864254
0      0.075663 -0.351308       9        1   3.231855
9      0.067542 -0.059640      10        1   3.051034, topic_info=     Category          Freq        Term         Total  loglift  logprob
656   Default  38087.000000      advice  38087.000000  30.0000  30.0000
34    Default  15697.000000    AskWomen  15697.000000  29.0000  29.0000
3501  Default  26124.000000     message  26124.000000  28.0000  28.0000
4441  Default  18421.000000      remove  18421.000000  27.0000  27.0000
3570  Default  18123.000000   moderator  18123.000000  26.0000  26.0000
...       ...           ...         ...           ...      ...      ...
763   Topic10    187.446207      answer   1139.604845   1.6847  -5.2196
3712  Topic10    281.103981         not  28449.548516  -1.1275  -4.8144
3774  Topic10    166.216702          oh    968.718914   1.7270  -5.3398
1790  Topic10    163.373487  definitely    881.714555   1.8039  -5.3571
5526  Topic10    180.802377         use   2747.256540   0.7687  -5.2557

[731 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
14        9  0.981254    Amazon
15        3  0.234121   America
15        8  0.758350   America
34        1  0.495496  AskWomen
34        5  0.504414  AskWomen
...     ...       ...       ...
5787      7  0.051711       yes
5787      8  0.004458       yes
5787      9  0.016048       yes
5787     10  0.348601       yes
5796      6  0.993996       yup

[1878 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 5, 7, 6, 3, 2, 8, 1, 10])

In [125]:
tf_vectorizer_awwomen = CountVectorizer(tokenizer= lambda x: x,
                                    lowercase = False,
                                    max_df = 0.5, 
                                    min_df = 10)
dtm_tf_awwomen = tf_vectorizer_awwomen.fit_transform(askwomen_women['body_clean'])



UsageError: Line magic function `%%time` not found.


In [126]:
%%time
lda_tf_women.fit(dtm_tf_awwomen)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
CPU times: user 53.2 s, sys: 37.2 s, total: 1min 30s
Wall time: 8min 16s


LatentDirichletAllocation(batch_size=2000, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=1)

In [128]:
%%time
pyLDAvis.sklearn.prepare(lda_tf_women, dtm_tf_awwomen, tf_vectorizer_awwomen)

CPU times: user 1min 56s, sys: 3.39 s, total: 1min 59s
Wall time: 8min 2s


/Users/valentina/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.095781 -0.075398       1        1  23.398235
0      0.024866 -0.136672       2        1  14.853653
1      0.113403  0.029325       3        1  11.529328
5      0.084553 -0.127128       4        1   9.517674
3      0.085549  0.005722       5        1   8.823418
6      0.087623  0.066833       6        1   7.869690
7     -0.493606 -0.039364       7        1   7.647452
8      0.000219 -0.183760       8        1   5.614023
2      0.044039  0.136967       9        1   5.516920
4     -0.042427  0.323475      10        1   5.229606, topic_info=      Category          Freq      Term         Total  loglift  logprob
268    Default  58172.000000  AskWomen  58172.000000  30.0000  30.0000
21210  Default  40494.000000         w  40494.000000  29.0000  29.0000
6039   Default  40362.000000  askwomen  40362.000000  28.0000  28.0000
16854  Default  41368.000000  question  41368.000000  27.0000  27.0000
21549  Default  41300.000000     woman  41300.000000  26.0000  26.0000
...        ...           ...       ...           ...      ...      ...
13730  Topic10   1838.120421      live  20142.377061   0.5568  -5.4514
6230   Topic10   1441.784586   awesome   3798.999152   1.9820  -5.6942
15245  Topic10   1539.296020       old  16264.894042   0.5932  -5.6288
13831  Topic10   1570.825652       lot  36303.405116  -0.1895  -5.6085
19970  Topic10   1539.882314     think  73959.667139  -0.9210  -5.6284

[732 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
155        5  0.998177    Amazon
162       10  0.998715  American
268        7  0.999992  AskWomen
528        6  0.001528     Black
528       10  0.997979     Black
...      ...       ...       ...
21723      2  0.153620     young
21723      3  0.000470     young
21723      5  0.001175     young
21723      6  0.000353     young
21723     10  0.047955     young

[1783 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 2, 6, 4, 7, 8, 9, 3, 5])

In [117]:
help(LatentDirichletAllocation)

Help on class LatentDirichletAllocation in module sklearn.decomposition.online_lda:

class LatentDirichletAllocation(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin)
 |  LatentDirichletAllocation(n_components=10, doc_topic_prior=None, topic_word_prior=None, learning_method='batch', learning_decay=0.7, learning_offset=10.0, max_iter=10, batch_size=128, evaluate_every=-1, total_samples=1000000.0, perp_tol=0.1, mean_change_tol=0.001, max_doc_update_iter=100, n_jobs=None, verbose=0, random_state=None)
 |  
 |  Latent Dirichlet Allocation with online variational Bayes algorithm
 |  
 |  .. versionadded:: 0.17
 |  
 |  Read more in the :ref:`User Guide <LatentDirichletAllocation>`.
 |  
 |  Parameters
 |  ----------
 |  n_components : int, optional (default=10)
 |      Number of topics.
 |  
 |  doc_topic_prior : float, optional (default=None)
 |      Prior of document topic distribution `theta`. If the value is None,
 |      defaults to `1 / n_components`.
 |      In [1]_, this is

### Don't Run The cod below this cell for now

In [49]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [50]:
# Create Dictionary
id2word = corpora.Dictionary(df_askmen['body_clean'])


id2word.filter_extremes(no_below=10, no_above=0.7, keep_n=50000)

In [51]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in df_askmen['body_clean']]
# corpus = [id2word.doc2bow(text) for text in tokenized_bigram]

tfidf = gensim.models.TfidfModel(corpus)
# corpus_tfidf = tfidf[corpus]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 4), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2)]]


In [52]:
df_askmen['body_clean'].iloc[0]


['never',
 'meet',
 'attractive',
 'woman',
 'single',
 'time',
 'sleep',
 'offer',
 'friend',
 'acquaintance',
 'stranger',
 'try',
 'find',
 'attractive',
 'single',
 'female',
 'friend',
 'want',
 'fuck',
 'awesome',
 'think',
 'creepy',
 'weird',
 'think',
 'pretty',
 'normal',
 'men',
 'woman',
 'absolutely',
 'friend',
 'aware',
 'friend',
 'fuck',
 'want']

In [53]:
print(id2word[0])
print(id2word[1])

absolutely
acquaintance


In [ ]:
from gensim.test.utils import datapath

In [54]:
%%time
if True:
    # Build LDA model
    # read this for distributed LDA: https://radimrehurek.com/gensim/models/lda_dispatcher.html
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           passes=5,
                                           workers=8,
                                           per_word_topics=True)

In [ ]:

# Save model to disk.
temp_file = datapath("model")
lda.save(temp_file)
# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [55]:
# Print the Keyword in the 10 topics
from pprint import pprint
pprint(lda_model.print_topics(num_topics =30, num_words=20))
doc_lda = lda_model[corpus]

[(0,
  '0.088*"question" + 0.063*"computer" + 0.043*"post" + 0.039*"read" + '
  '0.037*"actual" + 0.034*"not" + 0.030*"reason" + 0.030*"filter" + '
  '0.029*"message" + 0.023*"automatic" + 0.021*"answer" + 0.021*"remove" + '
  '0.020*"title" + 0.020*"delete" + 0.020*"word" + 0.019*"failure" + '
  '0.019*"recognize" + 0.019*"sidebar" + 0.019*"number" + 0.018*"appeal"'),
 (1,
  '0.078*"frequently" + 0.056*"r" + 0.047*"ask" + 0.024*"wiki" + 0.023*"hi" + '
  '0.022*"nope" + 0.021*"page" + 0.020*"similar" + 0.019*"consult" + '
  '0.018*"index" + 0.017*"Question" + 0.016*"search" + 0.016*"question" + '
  '0.015*"AskMen" + 0.013*"find" + 0.013*"thank" + 0.012*"post" + '
  '0.011*"remove" + 0.010*"sidebar" + 0.010*"step"'),
 (2,
  '0.012*"marry" + 0.010*"married" + 0.009*"marriage" + 0.009*"not" + '
  '0.009*"life" + 0.008*"happy" + 0.007*"want" + 0.007*"divorce" + '
  '0.006*"sure" + 0.006*"like" + 0.006*"woman" + 0.006*"people" + 0.006*"year" '
  '+ 0.005*"medium" + 0.005*"relationship" + 0.

In [56]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, 
                                     texts= df_askmen['body_clean'], 
                                     dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5754798958962317


In [57]:
# Visualize the topics
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

D:\Anaconda3\envs\fastai\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
